In [28]:
import pandas as pd 

In [29]:
df = pd.read_csv('/Users/daeun/Desktop/maestro_python/diabetes.csv')

In [30]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### 1. 데이터 전처리

In [32]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

### 2. train/test 데이터 세트로 분리

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [35]:
# Outcome 열 = 결과, 나머지 = 데이터

In [36]:
#df.iloc[행,열]
#df.iloc[: , :-1]
#      전체  전체 바로 앞

In [37]:
#df.iloc[행,열]
df.iloc[: , -1]
df['Outcome']

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [38]:
#결측치 확인
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [41]:
X = df.iloc[ : , :-1]
y = df['Outcome']

### 2. Train/Test Set 으로 분리

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X , y , test_size=0.2, random_state=1, stratify=y
)

In [62]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

### 3. 모델을 생성하고 학습

In [65]:
from sklearn.tree import DecisionTreeClassifier

In [66]:
# 모델 객체 생성
dtc = DecisionTreeClassifier(random_state=1)

In [67]:
#학습
dtc.fit(X , y)

DecisionTreeClassifier(random_state=1)

### 4. 학습된 모델로 예측

In [68]:
pred = dtc.predict(X_test)

### 5. 예측값과 실제값을 비교하여 모델 성능을 평가 (정확도 산출)

In [69]:
from sklearn.metrics import accuracy_score

In [70]:
accuracy_score(y_test, pred)  #정확도

1.0

### 3-5에서 진행했던 과정과 하이퍼 파라미터 튜닝을 동시에
### => GridSearchCV !!

In [71]:
from sklearn.model_selection import GridSearchCV

In [72]:
params = {
    'max_depth' : [2,4,6],
    'min_samples_split' : [2,3,4]
}

In [73]:
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

#grid파라미터로 학습
grid_dtc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'max_depth': [2, 4, 6],
                         'min_samples_split': [2, 3, 4]})

In [74]:
#최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [75]:
# 최고 정확도
grid_dtc.best_score_

0.7573170731707317

In [76]:
# 최적 파라미터, 최고 정확도로 예측
grid_pred = grid_dtc.best_estimator_.predict(X_test)

# 예측값과 실제값의 비교로 모델 성능 평가 - 정확도 산출
accuracy_score(y_test, grid_pred)

0.7532467532467533

In [77]:
pred.shape

(154,)

- 실제 값 하나를 주고 당뇨병인지 아닌지 여부를 확인

In [95]:
import numpy as np 

estimator = grid_dtc.best_estimator_

test_data = [4,110,92,0,0,37.6,0.191,30]
# predict = dtc.predict(test_data) => 그냥 넣으면 에러남. 차원이 안맞기 때문!

predict = estimator.predict(np.array(test_data).reshape(1,8))

In [96]:
predict

array([0])

In [97]:
predict[0]

0

In [98]:
estimator.predict([test_data])[0]

0